In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# System
import math
import os
import sys

# Data
import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

# Util
from six.moves import urllib
from sklearn.model_selection import train_test_split

# MachineLearning
try:
  # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers

tf.compat.v1.enable_eager_execution()

pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 15

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
train_data = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')

In [ ]:
test_data = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
test_data.info()
test_data.describe()

In [ ]:
train_data.info()
train_data.head()

In [ ]:
train_data.describe()
train_data.describe(include=['O'])

In [ ]:
train_data['MSSubClass'] = train_data['MSSubClass'].apply(int)
train_data['OverallQual'] = train_data['OverallQual'].apply(int)
train_data['OverallCond'] = train_data['OverallCond'].apply(int)

In [ ]:
label = 'SalePrice'
numeric_cols = list(set(train_data.describe().columns) - set([label]))
categorical_cols = list(set(train_data.columns) - set(numeric_cols) - set([label]))
print(len(numeric_cols))
print(len(categorical_cols))

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(70,60))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train_data[numeric_cols + [label]].astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
x = pd.DataFrame(abs(train_data.corr()[label]).sort_values(ascending = False))

x = x[x[label] >= 0.45]

x.index

features = list(set(x.index) - set(['GarageYrBlt', 'MasVnrArea']))

features

In [ ]:
train_data[features].info()

In [ ]:
print(categorical_cols)

train_data[['BsmtQual']].describe(include=["O"])

In [ ]:
train_data[[label]].plot.hist(bins=10)

In [ ]:
for col in categorical_cols[0:1]:
    print(col)
    for value in train_data[col].unique():
        df = train_data[train_data[col] == value][label]
        df.plot.hist(subplots=True)

In [ ]:
print(categorical_cols)


In [ ]:
fname = "BsmtQual"

train_data[[label]].plot.hist(bins=20)
train_data[[fname]].describe(include=["O"])
pal = {1:"seagreen", 0:"gray"}
g = sns.FacetGrid(train_data,size=5, col=fname, margin_titles=True,
                  palette=pal)
g = g.map(plt.hist, "SalePrice", edgecolor = 'white', bins=20);
g.fig.suptitle("SalePrice and " + fname, size = 25)
plt.subplots_adjust(top=0.90)

In [ ]:
train_data[["OverallQual", label]].groupby("OverallQual", as_index=False).mean().sort_values(by=label)

------------------------------------------------------------------------------------

In [ ]:
for feature in ['FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'OverallQual']:
    train_data[feature] = train_data[feature].apply(str)

In [ ]:
NUMERIC_COLUMNS = ['1stFlrSF', 'GarageArea', 'GrLivArea', 'TotalBsmtSF', 'YearBuilt', 'YearRemodAdd']
CATEGORICAL_COLUMNS = ['Neighborhood', 'ExterQual', 'HouseStyle', 'SaleCondition', 'KitchenQual', 'CentralAir', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'OverallQual']
# LABEL = 'SalePrice'

# NUMERIC_COLUMNS = ['1stFlrSF', 'GrLivArea', 'YearBuilt']
# CATEGORICAL_COLUMNS = ['SaleCondition', 'Neighborhood', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'OverallQual']
LABEL = 'SalePrice'

In [ ]:
# maxi = train_data[label].max()
# mini = train_data[label].min()
for feature in NUMERIC_COLUMNS:
    train_data[feature] = train_data[feature].apply(lambda x : np.log(x+1))
train_data[label] = np.log(train_data[label])

In [ ]:
dftrain, dfeval = train_test_split(train_data[CATEGORICAL_COLUMNS + NUMERIC_COLUMNS + [LABEL]], test_size=0.3)

ytrain = dftrain.pop(LABEL)
yeval = dfeval.pop(LABEL)

dftrain.info()


In [ ]:
feature_columns = []
categorical_feature_columns = []
crossed_feature_columns = []
numeric_feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    categorical_feature_columns.append(
        feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                feature_name, 
                vocabulary
            )
        )
    )

epsilon = 0.1

for feature_name in NUMERIC_COLUMNS:
    numeric_feature_columns.append(
        tf.feature_column.numeric_column(
            feature_name, 
            dtype=tf.float32,
#             normalizer_fn=lambda val: (val - dftrain.mean()[feature_name]) / (epsilon + dftrain.std()[feature_name])
#             normalizer_fn=lambda val: np.log(val)
        )
    )
    
# crossed_feature = feature_column.crossed_column([tf.feature_column.categorical_column_with_vocabulary_list(
#                 'SibSp', 
#                 vocabulary
#             ), tf.feature_column.categorical_column_with_vocabulary_list(
#                 'Parch', 
#                 vocabulary
#             )], hash_bucket_size=15)
# crossed_feature = feature_column.indicator_column(crossed_feature)
# crossed_feature_columns.append(crossed_feature)

# age_buckets = feature_column.bucketized_column(tf.feature_column.numeric_column(
#             feature_name, 
#             dtype=tf.float32,
#             normalizer_fn=lambda val: (val - dftrain.mean()[feature_name]) / (epsilon + dftrain.std()[feature_name])
#         ), boundaries=[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70])

# feature_columns.append(age_buckets)
feature_columns.extend(categorical_feature_columns) 
# feature_columns.extend(crossed_feature_columns)
feature_columns.extend(numeric_feature_columns)
print(len(feature_columns))

In [ ]:
# Config
BATCH_SIZE = 32
NUM_EPOCHS = 50

NUM_TRAINING_STEPS = 100


In [ ]:
# InputFunctors
def df_to_dataset(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    # Dataset needs to be re-assigned
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds

def testdf_to_dataset(data_df, num_epochs=10, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(dict(data_df))
    # Dataset needs to be re-assigned
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds

train_ds = df_to_dataset(dftrain, ytrain, num_epochs=NUM_EPOCHS, shuffle=True, batch_size=BATCH_SIZE)
eval_ds = df_to_dataset(dfeval, yeval, num_epochs=1, shuffle=False, batch_size=BATCH_SIZE)
# test_ds = testdf_to_dataset(test_data, num_epochs=1, batch_size=BATCH_SIZE)

ds = df_to_dataset(dftrain, ytrain, batch_size=5)
for feature_batch, label_batch in ds.take(1):
    print('Length of batch: ', len(label_batch))
    print('Number of features: ', len(feature_batch))
    print('Some feature keys:', list(feature_batch.keys()))
    print('A batch of Labels:', label_batch.numpy())
    print()

# InspectInput of model
def demo(example_batch, feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

example_batch = next(iter(ds))[0]
demo(example_batch, feature_columns)

In [ ]:
def make_input_fn(data_df, label_df, n_epochs=None, shuffle=True, batch_size=32):
    def input_fn():
        return df_to_dataset(data_df, label_df, n_epochs, shuffle, batch_size)
    return input_fn

def make_test_input_fn(data_df, n_epochs=None, batch_size=32):
    def input_fn():
        return testdf_to_dataset(data_df, n_epochs, batch_size)
    return input_fn

train_input_fn = make_input_fn(dftrain, ytrain, n_epochs=NUM_EPOCHS, shuffle=True, batch_size=BATCH_SIZE)
eval_input_fn = make_input_fn(dfeval, yeval, n_epochs=1, shuffle=False, batch_size=BATCH_SIZE)

In [ ]:
linear_est = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001))
linear_est.train(train_input_fn, steps=10000)
scores = linear_est.evaluate(eval_input_fn)
print('scores', scores)

In [ ]:
for feature in ['FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'OverallQual']:
    test_data[feature] = test_data[feature].apply(str)

for feature in NUMERIC_COLUMNS:
    test_data[feature] = test_data[feature].apply(lambda x : np.log(x+1))

# test_ds = testdf_to_dataset(test_data[NUMERIC_COLUMNS + CATEGORICAL_COLUMNS], num_epochs=1, batch_size=BATCH_SIZE)
test_input_fn = make_test_input_fn(test_data[NUMERIC_COLUMNS + CATEGORICAL_COLUMNS], n_epochs=1, batch_size=BATCH_SIZE)

In [ ]:
yeval.describe()

In [ ]:
predictions = linear_est.predict(test_input_fn)

In [ ]:
for item in predictions:
    print(np.exp(item['predictions'][0]))